In [1]:
import sys
sys.path.insert(1, '../')

In [10]:
import torch
import torch.nn as nn
from yolov2 import YOLOv2D19
from iou import intersection_over_union

In [9]:
model = YOLOv2D19()
img = torch.rand(1,3,416,416).to(torch.device('cuda:0'))
out = model(img)
out.shape

torch.Size([1, 125, 13, 13])

In [175]:
class YoloLoss(nn.Module):
    def __init__(self, S=13, B=5, C=20):
        super().__init__()
        self.mse = nn.MSELoss(reduction="mean")
        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, (self.C+5)*self.B)
        batch_size = predictions.shape[0]
        print(predictions.shape)

        # x,y,w,h,c,classes
        
        ious = torch.zeros(batch_size, self.B, self.S, self.S, 1)
        for b in range(self.B):
            offset = b*25
            pbox_idx = slice(0+offset,4+offset)
            tbox_idx = slice(0,4)
            
            iou = intersection_over_union(
                predictions[..., pbox_idx], 
                target[..., tbox_idx]
            )

            iou = torch.ones(2, 13, 13, 1).unsqueeze(1)
            ious[:, b:b+1, ...] = iou

        _, best_box = torch.max(ious, dim=0)
        exists_box = target[..., 4:5]

        # coord loss

        best_box_offset = best_box*25
        best_box_idx = slice(0+best_box_offset, 4+best_box_offset)
        box_predictions = exists_box * predictions[..., best_box_idx]

        return best_box


In [176]:
def test(out):
    loss = YoloLoss()
    predictions = torch.tensor([[[[7.5, 7.5, 10., 10., 
                       1., 
                       1., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                       0., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                      ]*5]*13]*13]*2)
    target = torch.tensor([[[[7.5, 7.5, 10., 10., 
                       1., 
                       1., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                       0., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                      ]*1]*13]*13]*2)
    return loss(predictions, target)

In [177]:
test(out)

torch.Size([2, 13, 13, 125])


TypeError: only integer tensors of a single element can be converted to an index

In [181]:
predictions = torch.tensor([[[[7.5, 7.5, 10., 10., 
                       1., 
                       1., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                       0., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                      ]*5]*13]*13]*2)

In [185]:
ious = torch.zeros(2, 5, 13, 13, 1)
_, best_box = torch.max(ious, dim=1)

In [186]:
best_box.shape

torch.Size([2, 13, 13, 1])

In [ ]:
30 = 20 cls, 4 coord, 1 conf, 4 coord, 1 conf